In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import tarfile
import logging
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report, precision_score

!wget http://vis-www.cs.umass.edu/lfw/lfw-funneled.tgz 
tfile = tarfile.open("lfw-funneled.tgz", "r:gz")
tfile.extractall(".")


--2023-06-07 08:41:12--  http://vis-www.cs.umass.edu/lfw/lfw-funneled.tgz
Resolving vis-www.cs.umass.edu (vis-www.cs.umass.edu)... 128.119.244.95
Connecting to vis-www.cs.umass.edu (vis-www.cs.umass.edu)|128.119.244.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 243346528 (232M) [application/x-gzip]
Saving to: ‘lfw-funneled.tgz’

lfw-funneled.tgz    100%[===================>] 232.07M   107MB/s    in 2.2s    

2023-06-07 08:41:14 (107 MB/s) - ‘lfw-funneled.tgz’ saved [243346528/243346528]



In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

X_train, X_test, y_train, y_test = train_test_split(lfw_people.images, lfw_people.target, test_size=0.2, random_state=42)

X_train = X_train / 255.0
X_test = X_test / 255.0


In [3]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(len(lfw_people.target_names)))

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [4]:
history = model.fit(X_train[:, :, :, np.newaxis], y_train, epochs=20, validation_data=(X_test[:, :, :, np.newaxis], y_test))

test_loss, test_acc = model.evaluate(X_test[:, :, :, np.newaxis], y_test)
predictions = model.predict(X_test[:, :, :, np.newaxis])
predicted_labels = np.argmax(predictions, axis=1)

target_names = lfw_people.target_names
classification_rep = classification_report(y_test, predicted_labels, target_names=target_names)
precision = precision_score(y_test, predicted_labels, average=None)
macro_precision = precision_score(y_test, predicted_labels, average='macro')


Epoch 1/20
33/33 [==============================] - 3s 56ms/step - loss: 1.7773 - accuracy: 0.3893 - val_loss: 1.6215 - val_accuracy: 0.4612
Epoch 2/20
33/33 [==============================] - 2s 48ms/step - loss: 1.7141 - accuracy: 0.3990 - val_loss: 1.6108 - val_accuracy: 0.4612
Epoch 3/20
33/33 [==============================] - 1s 42ms/step - loss: 1.7152 - accuracy: 0.3990 - val_loss: 1.6329 - val_accuracy: 0.4612
Epoch 4/20
33/33 [==============================] - 1s 41ms/step - loss: 1.7144 - accuracy: 0.3990 - val_loss: 1.5925 - val_accuracy: 0.4612
Epoch 5/20
33/33 [==============================] - 1s 42ms/step - loss: 1.7128 - accuracy: 0.3990 - val_loss: 1.6130 - val_accuracy: 0.4612
Epoch 6/20
33/33 [==============================] - 1s 44ms/step - loss: 1.7083 - accuracy: 0.3990 - val_loss: 1.5940 - val_accuracy: 0.4612
Epoch 7/20
33/33 [==============================] - 2s 71ms/step - loss: 1.7128 - accuracy: 0.3990 - val_loss: 1.5956 - val_accuracy: 0.4612
Epoch 8/20
33

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [6]:
print("Classification Report:")
print(classification_rep)
print("Macro Average Precision:", macro_precision)

Classification Report:
                   precision    recall  f1-score   support

     Ariel Sharon       0.00      0.00      0.00        11
     Colin Powell       0.00      0.00      0.00        47
  Donald Rumsfeld       0.00      0.00      0.00        22
    George W Bush       0.46      1.00      0.63       119
Gerhard Schroeder       0.00      0.00      0.00        19
      Hugo Chavez       0.00      0.00      0.00        13
       Tony Blair       0.00      0.00      0.00        27

         accuracy                           0.46       258
        macro avg       0.07      0.14      0.09       258
     weighted avg       0.21      0.46      0.29       258

Macro Average Precision: 0.06589147286821705
